**Import librarys**

In [1]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import os
import cv2
from sklearn.metrics import classification_report

**Check torch version**

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))
print("Antal CPU-kärnor:", os.cpu_count())

**Check Ultralytics settings**

In [4]:
print(settings)

JSONDict("C:\Users\shool\AppData\Roaming\Ultralytics\settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "C:\\Users\\shool\\Dev\\Codenames-Board\\modelv12\\datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "0847b39e9ea20ed4a1701ad224a7b4090217127da71fb1cbe1040083b7cf976f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}


**Download Dataset**

In [5]:
from roboflow import Roboflow

# dotenv.load_dotenv()
# settings.reset()

# ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
# PROJECT_ID = os.getenv("PROJECT_OBJECT_DETECTION")
# VERSION = os.getenv("OBJECT_DETECTION_MODEL_VERSION")

# def download_dataset(PROJECT_ID, version, yolov):
#   rf = Roboflow(api_key=ROBOFLOW_API_KEY)
#   project = rf.workspace().project(PROJECT_ID)
# #   version = project.version(version)
#   dataset = project.version(version).download(yolov)
#   return dataset

# dataset = download_dataset(PROJECT_ID, 9, yolov="yolov11")



https://docs.ultralytics.com/modes/train/#train-settings

**Train model**

In [2]:
def model_training(data):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO("yolo11n.pt")  # Laddar in förtränad modell
    model.to(device)
    results = model.train(
        data=data, 
        epochs=100, 
        imgsz=640, 
        batch=8,
        workers=os.cpu_count(),
        exist_ok=True
        )
    return results

results = model_training(data="modelv12.yaml")


New https://pypi.org/project/ultralytics/8.3.28 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=modelv12.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=12, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning C:\Users\shool\Dev\Codenames-Board\modelv12\datasets\labels\train... 141 images, 0 backgrounds, 0 corrupt: 100%|██████████| 141/141 [00:00<00:00, 365.12it/s]

train: New cache created: C:\Users\shool\Dev\Codenames-Board\modelv12\datasets\labels\train.cache



val: Scanning C:\Users\shool\Dev\Codenames-Board\modelv12\datasets\labels\val.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      1.42G     0.5474     0.4042     0.9098         14        640: 100%|██████████| 18/18 [00:02<00:00,  7.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  8.21it/s]

                   all         12        134      0.997          1      0.995      0.913



100 epochs completed in 0.119 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.21  Python-3.11.10 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  7.19it/s]


                   all         12        134      0.998          1      0.995      0.916
               class_0         12         69      0.999          1      0.995      0.972
               class_1         12         65      0.996          1      0.995       0.86
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train


**Validate model**

In [3]:
def model_validation():
    model = YOLO("runs/detect/train/weights/best.pt")
    metrics = model.val(
        data="modelv12.yaml", 
        imgsz=640, 
        batch=16, 
        conf=0.8, 
        iou=0.6, 
        device="0",
        dnn=True,
        exist_ok=True,
        save_json=True)
    return metrics


metrics = model_validation()
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)
print(metrics.box.maps)


Ultralytics 8.3.21  Python-3.11.10 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\shool\Dev\Codenames-Board\modelv12\datasets\labels\val.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:07<00:00,  7.51s/it]


                   all         12        134          1          1      0.995      0.929
               class_0         12         69          1          1      0.995      0.979
               class_1         12         65          1          1      0.995      0.879
Speed: 0.6ms preprocess, 10.4ms inference, 0.0ms loss, 2.5ms postprocess per image
Saving runs\detect\val\predictions.json...
Results saved to runs\detect\val
0.9294441006932447
0.995
0.995
[     0.9795     0.87939]


- **Box(P):** Precision för objektidentifiering, som anger hur många av modellens positiva förutsägelser som var korrekta.
- **R:** Recall, eller hur många av de verkliga objekten som modellen korrekt identifierade.
- **mAP@0.5:** Mean Average Precision vid en tröskel på 0.5, vilket är ett mått på hur bra modellen är på att hitta och korrekt avgränsa objekten.
- **mAP@0.5:0.95:** mAP beräknat över flera trösklar (från 0.5 till 0.95) för att ge en mer generell bild av modellens prestanda.
Denna rapport är mer omfattande än en klassisk klassificeringsrapport och fokuserar på modellens prestanda för objektidentifiering snarare än ren klassificering.

**Prediction**

In [5]:
def predict_image(image_path, conf=0.8):

    model = YOLO("runs/detect/train/weights/best.pt")
    
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    results = model.predict(
        img, 
        conf=conf, 
        stream=True,
        save=True
    )
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = [int(coord.item()) for coord in box.xyxy[0]]
            label = int(box.cls.item())
            confidence = box.conf.item()
            
            # Rita bounding boxen på bilden
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Label: {label}, Conf: {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


test_image = os.path.join(os.getcwd(), "test", "board_018.png")

predict_image(test_image)





0: 416x640 25 class_0s, 25 class_1s, 23.9ms
Speed: 4.1ms preprocess, 23.9ms inference, 4.6ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict
